This is the pop_density geohash 3 & 5 dataset for 2020 for ID  
Please run the file by 3 months  
It is set 20200101 - 20200331 for now, please change while submitting jobs  
Please remember to check logging information to fillin data missing dates  

In [1]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.autopandas=True
%config SqlMagic.displaylimit=200

In [2]:
%sql trino://localhost:9090/cuebiq/

'Connected: @cuebiq/'

In [3]:
pip install geohash2

Note: you may need to restart the kernel to use updated packages.


In [4]:
import gc
import os
import time
import logging
import geohash2
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from trino.dbapi import connect 

In [5]:
# SQL engine
class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

    def read_sql_chunked(self, query: str, chunksize: int = 10000):
        return pd.read_sql(query, self.engine, chunksize=chunksize)

sql_engine = TrinoEngine()

In [6]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}
pe_dl_table = f"{schema_name['cda']}.device_location_uplevelled"

In [7]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Database connection setup
output_schema_name = 'pop_density20'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

start_time = time.time()  # Start timing

# Define the input parameters
country_code = 'ID'
start_date = 20200101
end_date = 20200331

# Define the input schema and table name
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}
pe_dl_table = f"{schema_name['cda']}.device_location_uplevelled"

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')


In [8]:
# For ID - should be working

# Function to insert data with retry mechanism
def insert_data_with_retry(df, table_name, con, retries=3, delay=5):
    for attempt in range(retries):
        try:
            df.to_sql(
                table_name, 
                con, 
                index=False, 
                if_exists="append", 
                method="multi"
            )
            logging.info(f"Inserted data into table {table_name}")
            break
        except Exception as e:
            logging.error(f"Attempt {attempt+1} failed with error: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                logging.error(f"Failed to insert data into table {table_name} after {retries} attempts")

# Loop through each day from start_date to end_date
current_date = start_date_dt
while current_date <= end_date_dt:
    try:
        # Calculate the lookback and lookahead dates
        lookback_date = current_date - timedelta(days=1)
        lookahead_date = current_date + timedelta(days=35)
        
        # Format dates for the SQL query
        formatted_lookback_date = lookback_date.strftime('%Y%m%d')
        formatted_current_date = current_date.strftime('%Y%m%d')
        formatted_lookahead_date = lookahead_date.strftime('%Y%m%d')
        
        # Construct the SQL query
        query = f"""
        SELECT 
            cuebiq_id, 
            event_zoned_datetime, 
            processing_date,
            lat,
            lng
        FROM {pe_dl_table}
        WHERE 
            processing_date BETWEEN {formatted_lookback_date} AND {formatted_lookahead_date}
            AND country_code = '{country_code}' 
            AND event_zoned_datetime IS NOT NULL
            AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
            AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) = date_parse('{formatted_current_date}', '%Y%m%d')
        """
        
        logging.info(f"Executing SQL query for date {formatted_current_date}")
        
        pe_dl_table_gen = sql_engine.read_sql_chunked(query)
        
        # Convert the generator to a DataFrame
        chunks = [chunk for chunk in pe_dl_table_gen]
        if chunks:
            pe_dl_table_df = pd.concat(chunks, ignore_index=True)
            
            # Calculate geohashes
            pe_dl_table_df['geohash5'] = pe_dl_table_df.apply(lambda row: geohash2.encode(row['lat'], row['lng'], precision=5), axis=1)
            pe_dl_table_df['geohash3'] = pe_dl_table_df.apply(lambda row: geohash2.encode(row['lat'], row['lng'], precision=3), axis=1)
            
            # Aggregate data for geohash5
            aggregated_data_5 = pe_dl_table_df.groupby('geohash5').agg(
                no_of_points=('geohash5', 'size'),
                no_of_unique_users=('cuebiq_id', 'nunique')
            ).reset_index()

            # Filter rows with no_of_unique_users > 10
            filtered_data_5 = aggregated_data_5[aggregated_data_5['no_of_unique_users'] > 10].copy()
            
            # Add the local_date column
            filtered_data_5.loc[:, 'local_date'] = formatted_current_date
            
            # Insert filtered aggregated data for geohash5 into SQL table
            if not filtered_data_5.empty:
                table_name_agg5 = f"pd_{country_code.lower()}_{formatted_current_date}_agg5"
                insert_data_with_retry(filtered_data_5, table_name_agg5, con)
            
            # Aggregate data for geohash3
            aggregated_data_3 = pe_dl_table_df.groupby('geohash3').agg(
                no_of_points=('geohash3', 'size'),
                no_of_unique_users=('cuebiq_id', 'nunique')
            ).reset_index()
            
            # Filter rows with no_of_unique_users > 10
            filtered_data_3 = aggregated_data_3[aggregated_data_3['no_of_unique_users'] > 10].copy()
            
            # Add the local_date column
            filtered_data_3.loc[:, 'local_date'] = formatted_current_date
            
            # Insert filtered aggregated data for geohash3 into SQL table
            if not filtered_data_3.empty:
                table_name_agg3 = f"pd_{country_code.lower()}_{formatted_current_date}_agg3"
                insert_data_with_retry(filtered_data_3, table_name_agg3, con)
        
    except Exception as e:
        logging.error(f"Error while processing data for date {formatted_current_date}: {e}")
    
    # Move to the next day
    current_date += timedelta(days=1)

logging.info("Data extraction, aggregation, and saving completed.")

end_time = time.time()  # End timing

# Calculate and print the total time taken
total_time = end_time - start_time
logging.info(f"Total time taken: {total_time} seconds")

2024-06-07 14:54:36,458 - INFO - Executing SQL query for date 20190714
2024-06-07 15:00:39,752 - INFO - Inserted data into table pd_id_20190714_agg5
2024-06-07 15:00:43,344 - INFO - Inserted data into table pd_id_20190714_agg3
2024-06-07 15:00:43,345 - INFO - Data extraction, aggregation, and saving completed.
2024-06-07 15:00:43,345 - INFO - Total time taken: 368.349977016449 seconds


In [9]:
# # Filling missing date gaps for certain dates

# import logging
# import time
# from datetime import datetime, timedelta
# import pandas as pd
# import geohash2
# from sqlalchemy import create_engine

# # Set up logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# # Database connection setup
# output_schema_name = 'pop_density'
# con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

# start_time = time.time()  # Start timing

# # Define the input parameters
# country_code = 'ID'
# date_list = ['20190719','20190731', '20190808', '20190817', '20191020', '20191021']

# # Define the input schema and table name
# schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}
# pe_dl_table = f"{schema_name['cda']}.device_location_uplevelled"

# # Function to insert data with retry mechanism
# def insert_data_with_retry(df, table_name, con, retries=3, delay=5):
#     for attempt in range(retries):
#         try:
#             df.to_sql(
#                 table_name, 
#                 con, 
#                 index=False, 
#                 if_exists="append", 
#                 method="multi"
#             )
#             logging.info(f"Inserted data into table {table_name}")
#             break
#         except Exception as e:
#             logging.error(f"Attempt {attempt+1} failed with error: {e}")
#             if attempt < retries - 1:
#                 time.sleep(delay)
#             else:
#                 logging.error(f"Failed to insert data into table {table_name} after {retries} attempts")

# # Loop through each date in the date_list
# for date_str in date_list:
#     try:
#         # Convert date string to datetime object
#         current_date = datetime.strptime(date_str, '%Y%m%d')
        
#         # Calculate the lookback and lookahead dates
#         lookback_date = current_date - timedelta(days=1)
#         lookahead_date = current_date + timedelta(days=35)
        
#         # Format dates for the SQL query
#         formatted_lookback_date = lookback_date.strftime('%Y%m%d')
#         formatted_current_date = current_date.strftime('%Y%m%d')
#         formatted_lookahead_date = lookahead_date.strftime('%Y%m%d')

#         # Construct the SQL query
#         query = f"""
#         SELECT 
#             cuebiq_id, 
#             event_zoned_datetime, 
#             processing_date,
#             lat,
#             lng
#         FROM {pe_dl_table}
#         WHERE 
#             processing_date BETWEEN {formatted_lookback_date} AND {formatted_lookahead_date}
#             AND country_code = '{country_code}' 
#             AND event_zoned_datetime IS NOT NULL
#             AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
#             AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) = date_parse('{formatted_current_date}', '%Y%m%d')
#         """
        
#         logging.info(f"Executing SQL query for date {formatted_current_date}")
        
#         pe_dl_table_gen = sql_engine.read_sql_chunked(query)
        
#         # Convert the generator to a DataFrame
#         chunks = [chunk for chunk in pe_dl_table_gen]
#         if chunks:
#             pe_dl_table_df = pd.concat(chunks, ignore_index=True)
            
#             # Calculate geohashes
#             pe_dl_table_df['geohash5'] = pe_dl_table_df.apply(lambda row: geohash2.encode(row['lat'], row['lng'], precision=5), axis=1)
#             pe_dl_table_df['geohash3'] = pe_dl_table_df.apply(lambda row: geohash2.encode(row['lat'], row['lng'], precision=3), axis=1)
            
#             # Aggregate data for geohash5
#             aggregated_data_5 = pe_dl_table_df.groupby('geohash5').agg(
#                 no_of_points=('geohash5', 'size'),
#                 no_of_unique_users=('cuebiq_id', 'nunique')
#             ).reset_index()

#             # Filter rows with no_of_unique_users > 10
#             filtered_data_5 = aggregated_data_5[aggregated_data_5['no_of_unique_users'] > 10].copy()
            
#             # Add the local_date column
#             filtered_data_5.loc[:, 'local_date'] = formatted_current_date
            
#             # Insert filtered aggregated data for geohash5 into SQL table
#             if not filtered_data_5.empty:
#                 table_name_agg5 = f"pd_{country_code.lower()}_{formatted_current_date}_agg5"
#                 insert_data_with_retry(filtered_data_5, table_name_agg5, con)
            
#             # Aggregate data for geohash3
#             aggregated_data_3 = pe_dl_table_df.groupby('geohash3').agg(
#                 no_of_points=('geohash3', 'size'),
#                 no_of_unique_users=('cuebiq_id', 'nunique')
#             ).reset_index()
            
#             # Filter rows with no_of_unique_users > 10
#             filtered_data_3 = aggregated_data_3[aggregated_data_3['no_of_unique_users'] > 10].copy()
            
#             # Add the local_date column
#             filtered_data_3.loc[:, 'local_date'] = formatted_current_date
            
#             # Insert filtered aggregated data for geohash3 into SQL table
#             if not filtered_data_3.empty:
#                 table_name_agg3 = f"pd_{country_code.lower()}_{formatted_current_date}_agg3"
#                 insert_data_with_retry(filtered_data_3, table_name_agg3, con)
        
#     except Exception as e:
#         logging.error(f"Error while processing data for date {formatted_current_date}: {e}")

# logging.info("Data extraction, aggregation, and saving completed.")

# end_time = time.time()  # End timing

# # Calculate and print the total time taken
# total_time = end_time - start_time
# logging.info(f"Total time taken: {total_time} seconds")


2024-06-07 15:03:51,092 - INFO - Executing SQL query for date 20190719
2024-06-07 15:09:50,247 - INFO - Inserted data into table pd_id_20190719_agg5
2024-06-07 15:09:53,109 - INFO - Inserted data into table pd_id_20190719_agg3
2024-06-07 15:09:53,110 - INFO - Executing SQL query for date 20190731
2024-06-07 15:16:15,481 - INFO - Inserted data into table pd_id_20190731_agg5
2024-06-07 15:16:18,405 - INFO - Inserted data into table pd_id_20190731_agg3
2024-06-07 15:16:18,405 - INFO - Executing SQL query for date 20190808
2024-06-07 15:22:49,021 - INFO - Inserted data into table pd_id_20190808_agg5
2024-06-07 15:22:52,046 - INFO - Inserted data into table pd_id_20190808_agg3
2024-06-07 15:22:52,047 - INFO - Executing SQL query for date 20190817
2024-06-07 15:30:57,505 - INFO - Inserted data into table pd_id_20190817_agg5
2024-06-07 15:31:00,979 - INFO - Inserted data into table pd_id_20190817_agg3
2024-06-07 15:31:00,980 - INFO - Executing SQL query for date 20191020
2024-06-07 15:42:25,4